In [1]:
import requests
import json
import math
import pandas as pd
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

In [2]:
def to_finish(round_id, final_round):
    if round_id == 1 and final_round == 7:
        return 68
    else:
        return 2 ** (final_round + 1 - round_id)


def convert(game, final_round):
    winner = list(filter(lambda x: not x["isWinner"], game["teams"]))
    # In 2018 there is an issue where isWinner is false for both teams. If that
    # is the case, then check scores manually.
    if len(winner) == 1:
        team = winner[0]
    else:
        score0 = game["teams"][0]["score"]
        score1 = game["teams"][1]["score"]
        if score0 > score1:
            team = game["teams"][1]
        else:
            team = game["teams"][0]
    teamName = team["nameFull"]
    seed = team["seed"]
    finish = to_finish(math.floor(game["bracketId"] / 100), final_round)
    return [teamName, seed, finish]


def get_champion(games, final_round):
    games.sort(key=lambda x: x["bracketId"], reverse=True)
    championship = games[0]
    team = list(filter(lambda x: x["isWinner"], championship["teams"]))[0]
    teamName = team["nameFull"]
    seed = team["seed"]
    finish = to_finish(math.floor(championship["bracketId"] / 100) + 1, final_round)
    return [teamName, seed, finish]

In [3]:
ids = [
    6200,  # 2022
    6124,  # 2021
    5761,  # 2019
    5489,  # 2018
    4761,  # 2017
    3921,  # 2016
]
url = "https://sdataprod.ncaa.com/?operationName=get_championship_by_id_ncaa&variables=%7B%22championshipId%22%3A{champ_id}%2C%22sportUrl%22%3A%22basketball-women%22%2C%22division%22%3A1%2C%22year%22%3A{year}%2C%22showUnstaged%22%3Afalse%2C%22staticTestEnv%22%3Anull%7D&extensions=%7B%22persistedQuery%22%3A%7B%22version%22%3A1%2C%22sha256Hash%22%3A%22f08bb9619f4e1b045d0c2f6fa82261fce5cf4e16bdd2e61f9d9f9772b9a40ec0%22%7D%7D"

In [4]:
def load_season(url):
    data = requests.get(url).json()
    year = data["data"]["championships"][0]["year"]
    games = data["data"]["championships"][0]["games"]
    games.sort(key=lambda x: x["bracketId"])
    final_round = math.floor(games[-1]["bracketId"] / 100)

    df = pd.DataFrame(
        list(map(lambda x: convert(x, final_round), games))
        + [get_champion(games, final_round)],
        columns=["team", "seed", "finish"],
    )
    df["year"] = year
    return df


df = pd.concat(list(map(lambda x: load_season(url.format(champ_id=x, year=0)), ids)))
df.reset_index(drop=True)
df = df[["year", "team", "seed", "finish"]]
df

,year,team,seed,finish
0,2022,University of the Incarnate Word,16,68
1,2022,DePaul University,11,68
2,2022,Mount St. Mary's University,16,68
3,2022,Florida State University,11,68
4,2022,Howard University,16,64
...,...,...,...,...
59,2016,Stanford University,4,8
60,2016,Oregon State University,2,4
61,2016,University of Washington,7,4
62,2016,Syracuse University,4,2


In [5]:
df.to_csv("recent_matches.csv", index=False)